In [51]:
%reload_ext autoreload
%autoreload 2

import nltk
nltk.download('stopwords')
nltk.download('punkt')
import string

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

## Plot
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import matplotlib as plt

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
#data_preprocessing
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk import word_tokenize
from string import punctuation
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import GridSearchCV
from bs4 import BeautifulSoup 
from nltk.tokenize import WordPunctTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
#from xgboost import XGBClassifier

##import dataset
dataset = pd.read_excel('train.xlsx')

##Fill missing values of Host with that link in the same row
for i in list(dataset[dataset['Host'].isnull() == True]['Host'].index):
    dataset.at[i,'Host'] = dataset.loc[i]['Link']
    
##Filling missing value of TRANS_CONV_TEXT with that of title
for i in list(dataset[dataset['TRANS_CONV_TEXT'].isnull() == True].index):
    dataset.at[i, 'TRANS_CONV_TEXT'] = dataset.loc[i]['Title']
##Converting 'Date(ET)' object to datetime
dataset['Date(ET)'] = pd.to_datetime(dataset['Date(ET)'], infer_datetime_format=True)    

##Converting Datetime to timestamp
dataset['Date(ET)'] = dataset[['Date(ET)']].apply(lambda x: x[0].timestamp(), axis = 1).astype(int)

##Dropping time(ET) and time(GMT)
dataset.drop(['Date(ET)', 'Time(ET)', 'time(GMT)'], axis = 1, inplace = True)

##Fill missing values of Title with that of TRANS_CONV_TEXT in the same row
for i in list(dataset[dataset['Title'].isnull() == True]['Title'].index):
    dataset.at[i,'Title'] = dataset.loc[i]['TRANS_CONV_TEXT']

## TEST Data
test = pd.read_csv('test.csv', encoding = 'utf-8')

##Fill missing values of Host with that link in the same row
for i in list(test[test['Host'].isnull() == True]['Host'].index):
    test.at[i,'Host'] = test.loc[i]['Link']
    
#Filling missing value of TRANS_CONV_TEXT with that of title
for i in list(test[test['TRANS_CONV_TEXT'].isnull() == True].index):
    test.at[i, 'TRANS_CONV_TEXT'] = test.loc[i]['Title']

test.at[441,'Date(ET)'] = test.loc[441, 'Time(ET)']

#Converting 'Date(ET)' object to datetime
test['Date(ET)'] = pd.to_datetime(test['Date(ET)'], infer_datetime_format=True)    

#Converting Datetime to timestamp
test['Date(ET)'] = test[['Date(ET)']].apply(lambda x: x[0].timestamp(), axis = 1).astype(int)

#Dropping time(ET) and time(GMT)
test.drop(['Date(ET)','Time(ET)', 'time(GMT)'], axis = 1, inplace = True)

#Fill missing values of Title with that of TRANS_CONV_TEXT in the same row
for i in list(test[test['Title'].isnull() == True]['Title'].index):
    test.at[i,'Title'] = test.loc[i]['TRANS_CONV_TEXT']

index = test['Index']
index = list(index)

test.drop(['Index'], axis = 1, inplace = True)

#feature_selection
y = dataset['Patient_Tag'].tolist()

dataset['Story'] = 'a'


for i in range(len(dataset)):
    dataset.at[i, 'Story'] = dataset['Source'][i] + ' ' + dataset['Host'][i] + ' ' + str(dataset['Link'][i]) + ' ' + dataset['Title'][i] + ' ' + dataset['TRANS_CONV_TEXT'][i]

dataset.drop(['Source', 'Host', 'Link', 'Title', 'TRANS_CONV_TEXT'], axis = 1, inplace = True)


test['Story'] = 'a'


for i in range(len(test)):
    test.at[i, 'Story'] = test['Source'][i] + ' ' + test['Host'][i] + ' ' + str(test['Link'][i]) + ' ' + test['Title'][i] + ' ' + test['TRANS_CONV_TEXT'][i]

test.drop(['Source', 'Host', 'Link', 'Title', 'TRANS_CONV_TEXT', 'Unnamed: 9'], axis = 1, inplace = True)

replace_space = re.compile('[/(){}\[\]\|@,;]')
bad_symbols = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def preprocess(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text 

dataset['Story'] = dataset['Story'].map(lambda x: preprocess(x))
test['Story'] = test['Story'].map(lambda x: preprocess(x))

data_train_list = dataset['Story'].tolist()
data_test_list = test['Story'].tolist()

all_data = data_train_list + data_test_list









[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [52]:
#Part 1
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
# Others
import string

from sklearn.manifold import TSNE


### Create sequence
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(dataset['Story'])
sequences = tokenizer.texts_to_sequences(dataset['Story'])
data = pad_sequences(sequences, maxlen=50)

#test
sequences_test = tokenizer.texts_to_sequences(test['Story'])
x_test = pad_sequences(sequences_test, maxlen = 50)


## Network architecture
model = Sequential()
model.add(Embedding(20000, 100, input_length=50))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit the model
model.fit(data, dataset['Patient_Tag'], validation_split=0.3, epochs=25)




Train on 809 samples, validate on 348 samples
Epoch 1/25
809/809 [==============================] - 8s 10ms/step - loss: 0.5880 - acc: 0.7787 - val_loss: 0.4995 - val_acc: 0.7902
Epoch 2/25
809/809 [==============================] - 3s 4ms/step - loss: 0.3885 - acc: 0.8096 - val_loss: 0.3455 - val_acc: 0.8190
Epoch 3/25
809/809 [==============================] - 3s 4ms/step - loss: 0.1398 - acc: 0.9604 - val_loss: 0.3453 - val_acc: 0.8563
Epoch 4/25
809/809 [==============================] - 3s 4ms/step - loss: 0.0524 - acc: 0.9827 - val_loss: 0.3322 - val_acc: 0.8448
Epoch 5/25
809/809 [==============================] - 3s 3ms/step - loss: 0.0284 - acc: 0.9938 - val_loss: 0.4050 - val_acc: 0.8563
Epoch 6/25
809/809 [==============================] - 3s 3ms/step - loss: 0.0074 - acc: 0.9988 - val_loss: 0.4632 - val_acc: 0.8563
Epoch 7/25
809/809 [==============================] - 3s 4ms/step - loss: 0.0049 - acc: 1.0000 - val_loss: 0.5587 - val_acc: 0.8592
Epoch 8/25
809/809 [=========

In [53]:

y_pred = model.predict([x_test], batch_size=32, verbose=0)
y_test = []
for i in range(len(y_pred)):
  if y_pred[i][0] > 0.7:
    y_test.append(1)
  else:
    y_test.append(0)

submission = pd.DataFrame()
submission['Index'] = index
submission['Patient_Tag'] = y_test
submission.to_csv("lstm_.csv", index=None)
submission.head()

,Index,Patient_Tag
0,1,0
1,2,1
2,3,0
3,4,1
4,5,0


In [54]:
# Part 2: LSTM with CNN
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

from sklearn.manifold import TSNE


def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(vocabulary_size, 100, input_length=50))
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    model_conv.add(LSTM(100))
    model_conv.add(Dense(1, activation='sigmoid'))
    model_conv.compile(loss='binary_crossentropy', optimizer='adam',    metrics=['accuracy'])
    return model_conv

model_conv = create_conv_model()
model_conv.fit(data, dataset['Patient_Tag'], validation_split=0.3, epochs = 15)



Train on 809 samples, validate on 348 samples
Epoch 1/15
809/809 [==============================] - 7s 9ms/step - loss: 0.5426 - acc: 0.7936 - val_loss: 0.4814 - val_acc: 0.7902
Epoch 2/15
809/809 [==============================] - 2s 2ms/step - loss: 0.3841 - acc: 0.7936 - val_loss: 0.3671 - val_acc: 0.7902
Epoch 3/15
809/809 [==============================] - 2s 2ms/step - loss: 0.1911 - acc: 0.9098 - val_loss: 0.3460 - val_acc: 0.8477
Epoch 4/15
809/809 [==============================] - 2s 2ms/step - loss: 0.0519 - acc: 0.9876 - val_loss: 0.5637 - val_acc: 0.8391
Epoch 5/15
809/809 [==============================] - 2s 2ms/step - loss: 0.0084 - acc: 0.9988 - val_loss: 0.6467 - val_acc: 0.8132
Epoch 6/15
809/809 [==============================] - 2s 2ms/step - loss: 0.0015 - acc: 1.0000 - val_loss: 0.7013 - val_acc: 0.8276
Epoch 7/15
809/809 [==============================] - 2s 2ms/step - loss: 5.3152e-04 - acc: 1.0000 - val_loss: 0.7472 - val_acc: 0.8247
Epoch 8/15
809/809 [======

In [55]:
y_pred = model_conv.predict([x_test], batch_size=32, verbose=0)
y_test = []
for i in range(len(y_pred)):
  if y_pred[i][0] > 0.7:
    y_test.append(1)
  else:
    y_test.append(0)

submission = pd.DataFrame()
submission['Index'] = index
submission['Patient_Tag'] = y_test
submission.to_csv("lstm_cnn.csv", index=None)
submission.head()

,Index,Patient_Tag
0,1,0
1,2,1
2,3,0
3,4,1
4,5,0


In [60]:
# Part 3: LSTM+CNN+Glove Embedding

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.manifold import TSNE

# Extract word embeddings from Glove
embeddings_index = dict()
f = open('glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()


# Create a weight matrix
embedding_matrix = np.zeros((vocabulary_size, 50))
for word, index_ in tokenizer.word_index.items():
    if index_ > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index_] = embedding_vector

## create model
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 50, input_length=50, weights=[embedding_matrix], trainable=True))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(100))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit train data
model_glove.fit(data, dataset['Patient_Tag'], validation_split=0.4, epochs = 25)



Train on 694 samples, validate on 463 samples
Epoch 1/25
694/694 [==============================] - 7s 10ms/step - loss: 0.4448 - acc: 0.8026 - val_loss: 0.3989 - val_acc: 0.8359
Epoch 2/25
694/694 [==============================] - 1s 1ms/step - loss: 0.3445 - acc: 0.8386 - val_loss: 0.3803 - val_acc: 0.8337
Epoch 3/25
694/694 [==============================] - 1s 1ms/step - loss: 0.2950 - acc: 0.8674 - val_loss: 0.3773 - val_acc: 0.8099
Epoch 4/25
694/694 [==============================] - 1s 1ms/step - loss: 0.2534 - acc: 0.8804 - val_loss: 0.3605 - val_acc: 0.8726
Epoch 5/25
694/694 [==============================] - 1s 1ms/step - loss: 0.2146 - acc: 0.9121 - val_loss: 0.3477 - val_acc: 0.8553
Epoch 6/25
694/694 [==============================] - 1s 1ms/step - loss: 0.1655 - acc: 0.9308 - val_loss: 0.3636 - val_acc: 0.8445
Epoch 7/25
694/694 [==============================] - 1s 1ms/step - loss: 0.1320 - acc: 0.9424 - val_loss: 0.3526 - val_acc: 0.8488
Epoch 8/25
694/694 [=========

In [61]:
y_pred = model_glove.predict([x_test], batch_size=32, verbose=0)
y_test = []
for i in range(len(y_pred)):
  if y_pred[i][0] > 0.7:
    y_test.append(1)
  else:
    y_test.append(0)

submission = pd.DataFrame()
submission['Index'] = index
submission['Patient_Tag'] = y_test
submission.to_csv("lstm_cnn_glove.csv", index=None)
submission.head()

,Index,Patient_Tag
0,1,0
1,2,1
2,3,0
3,4,1
4,5,0
